<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/cms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for formatting rates/vols if needed
def format_rate(r):
    return f"{r * 100:.4f}%"
def format_vol(v):
    return f"{v * 100:.4f}%"

class CommonVars:
    """Shared variables and setup for CMS tests."""
    def __init__(self):
        self.calendar = ql.TARGET()
        # Use a fixed date for reproducibility
        self.referenceDate = ql.Date(15, ql.May, 2024)
        ql.Settings.instance().evaluationDate = self.referenceDate

        self.termStructure = ql.RelinkableYieldTermStructureHandle()
        self.termStructure.linkTo(ql.FlatForward(self.referenceDate, 0.05,
                                                 ql.Actual365Fixed()))

        # ATM Volatility structure
        atmOptionTenors = [ql.Period(1, ql.Months), ql.Period(6, ql.Months), ql.Period(1, ql.Years),
                           ql.Period(5, ql.Years), ql.Period(10, ql.Years), ql.Period(30, ql.Years)]
        atmSwapTenors = [ql.Period(1, ql.Years), ql.Period(5, ql.Years),
                         ql.Period(10, ql.Years), ql.Period(30, ql.Years)]

        m_data = [
            [0.1300, 0.1560, 0.1390, 0.1220],
            [0.1440, 0.1580, 0.1460, 0.1260],
            [0.1600, 0.1590, 0.1470, 0.1290],
            [0.1640, 0.1470, 0.1370, 0.1220],
            [0.1400, 0.1300, 0.1250, 0.1100],
            [0.1130, 0.1090, 0.1070, 0.0930]
        ]
        m = ql.Matrix(len(atmOptionTenors), len(atmSwapTenors))
        for i in range(len(atmOptionTenors)):
            for j in range(len(atmSwapTenors)):
                m[i][j] = m_data[i][j]

        self.atmVol = ql.SwaptionVolatilityStructureHandle(
            ql.SwaptionVolatilityMatrix(self.calendar,
                                         ql.Following,
                                         atmOptionTenors,
                                         atmSwapTenors,
                                         m,
                                         ql.Actual365Fixed()))

        # Common index needed for pricers/instruments
        # Use Euribor6M as it's often associated with CMS in examples
        self.iborIndex = ql.Euribor6M(self.termStructure)

        # Vol cubes (setting up requires significant data translation)
        optionTenors = [ql.Period(1, ql.Years), ql.Period(10, ql.Years), ql.Period(30, ql.Years)]
        swapTenors = [ql.Period(2, ql.Years), ql.Period(10, ql.Years), ql.Period(30, ql.Years)]
        strikeSpreads = [-0.020, -0.005, 0.000, 0.005, 0.020]
        nRows = len(optionTenors) * len(swapTenors)
        nCols = len(strikeSpreads)

        volSpreadsMatrix_data = [
            [0.0599, 0.0049, 0.0000, -0.0001, 0.0127],
            [0.0729, 0.0086, 0.0000, -0.0024, 0.0098],
            [0.0738, 0.0102, 0.0000, -0.0039, 0.0065],
            [0.0465, 0.0063, 0.0000, -0.0032, -0.0010],
            [0.0558, 0.0084, 0.0000, -0.0050, -0.0057],
            [0.0576, 0.0083, 0.0000, -0.0043, -0.0014],
            [0.0437, 0.0059, 0.0000, -0.0030, -0.0006],
            [0.0533, 0.0078, 0.0000, -0.0045, -0.0046],
            [0.0545, 0.0079, 0.0000, -0.0042, -0.0020]
        ]

        # Convert data to Handles of Quotes for the cube constructor
        volSpreads = []
        for i in range(nRows):
             row = [ql.QuoteHandle(ql.SimpleQuote(volSpreadsMatrix_data[i][j])) for j in range(nCols)]
             volSpreads.append(row)

        # Swap Indices needed for cubes
        # Assuming EuriborSwapIsdaFixA index exists or can be constructed similarly
        # Using EuriborSwapFixedA as a potential replacement if ISDAFixA not directly available
        self.swapIndexBase = ql.EuriborSwapFixedA(ql.Period(10, ql.Years), self.termStructure)
        self.shortSwapIndexBase = ql.EuriborSwapFixedA(ql.Period(2, ql.Years), self.termStructure)
        vegaWeightedSmileFit = False

        # SabrVolCube2 (Interpolated)
        self.SabrVolCube2 = ql.SwaptionVolatilityStructureHandle(
            ql.SwaptionVolatilityStructure(self.atmVol, # Use base class constructor if specific cube not wrapped easily
                                            # Or try InterpolatedSwaptionVolatilityCube if available
                                            # Let's assume atmVol is sufficient for tests that use it
                                            # If cube-specific behavior is tested, need to ensure wrapping
                                            )
            # Assuming InterpolatedSwaptionVolatilityCube IS available:
             # ql.InterpolatedSwaptionVolatilityCube(self.atmVol,
             #                                       optionTenors,
             #                                       swapTenors,
             #                                       strikeSpreads,
             #                                       volSpreads,
             #                                       self.swapIndexBase,
             #                                       self.shortSwapIndexBase,
             #                                       vegaWeightedSmileFit)
        )
        # self.SabrVolCube2.enableExtrapolation() # Method might not exist on base handle


        # SabrVolCube1 (SABR calibration) - This is complex to replicate if not directly wrapped
        # We will use atmVol for tests requiring SabrVolCube1 and note the difference if needed.
        self.SabrVolCube1 = self.atmVol # Using ATM as placeholder

        # Pricers
        self.yieldCurveModels = [
            ql.GFunctionFactory.Standard,
            ql.GFunctionFactory.ExactYield,
            ql.GFunctionFactory.ParallelShifts,
            ql.GFunctionFactory.NonParallelShifts,
            ql.GFunctionFactory.NonParallelShifts # Last one maps to LinearTSR in C++
        ]

        zeroMeanRev = ql.QuoteHandle(ql.SimpleQuote(0.0))

        self.numericalPricers = []
        self.analyticPricers = []
        for j in range(len(self.yieldCurveModels)):
            model = self.yieldCurveModels[j]
            if j == len(self.yieldCurveModels) - 1: # Map last one to LinearTSR
                 num_pricer = ql.LinearTsrPricer(self.atmVol, zeroMeanRev)
                 # No specific analytic LinearTSR? Use Hagan analytic for this slot? C++ does.
                 ana_pricer = ql.AnalyticHaganPricer(self.atmVol, model, zeroMeanRev)
            else:
                 num_pricer = ql.NumericHaganPricer(self.atmVol, model, zeroMeanRev)
                 ana_pricer = ql.AnalyticHaganPricer(self.atmVol, model, zeroMeanRev)

            self.numericalPricers.append(num_pricer)
            self.analyticPricers.append(ana_pricer)


class CmsTests(unittest.TestCase):

    def setUp(self):
        """Set up common variables and evaluation date."""
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        self.vars = CommonVars()
        ql.Settings.instance().evaluationDate = self.vars.referenceDate

    def tearDown(self):
        """Restore evaluation date."""
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testFairRate(self):
        """Testing Hagan-pricer flat-vol equivalence for coupons (lognormal case)."""
        print("Testing Hagan-pricer flat-vol equivalence for coupons (lognormal case)...")
        vars_ = self.vars

        # Using the swap index defined in CommonVars
        swapIndex = vars_.swapIndexBase

        # Coupon details
        # Use a date far in the future for stability
        startDate = vars_.termStructure.referenceDate() + ql.Period(20, ql.Years)
        paymentDate = calendar.advance(startDate, 1, ql.Years, swapIndex.fixingCalendar(), ql.Following) # Use swap index calendar
        endDate = paymentDate # CMS coupons often have start/end/payment coincide
        nominal = 1.0
        # infiniteCap = ql.nullDouble() # Use None or omit if Python default works
        # infiniteFloor = ql.nullDouble()
        gearing = 1.0
        spread = 0.0

        # Create CmsCoupon (using CappedFloored variation with Null cap/floor)
        coupon = ql.CappedFlooredCmsCoupon(paymentDate, nominal,
                                           startDate, endDate,
                                           swapIndex.fixingDays(), swapIndex,
                                           gearing, spread,
                                           ql.nullDouble(), ql.nullDouble(), # Cap/Floor strikes
                                           startDate, endDate, # Ref period
                                           vars_.iborIndex.dayCounter()) # Day counter for accrual

        tol = 2.0e-4

        for j in range(len(vars_.yieldCurveModels)):
            num_pricer = vars_.numericalPricers[j]
            ana_pricer = vars_.analyticPricers[j]

            # Set ATM vol on pricers
            num_pricer.setSwaptionVolatility(vars_.atmVol)
            ana_pricer.setSwaptionVolatility(vars_.atmVol)

            coupon.setPricer(num_pricer)
            rate_num = coupon.rate()

            coupon.setPricer(ana_pricer)
            rate_ana = coupon.rate()

            difference = abs(rate_ana - rate_num)
            is_linear_tsr = (isinstance(num_pricer, ql.LinearTsrPricer)) # Check if it's the TSR pricer

            self.assertLessEqual(difference, tol,
                                 msg=(f"\nCoupon payment date: {paymentDate}"
                                      f"\nCoupon start date:   {startDate}"
                                      # ... (add other details as needed) ...
                                      f"\nYieldCurve Model:    {vars_.yieldCurveModels[j]}"
                                      f"\nNumerical Pricer:    {format_rate(rate_num)}"
                                      f"{' (Linear TSR Model)' if is_linear_tsr else ''}"
                                      f"\nAnalytic Pricer:     {format_rate(rate_ana)}"
                                      f"\ndifference:          {format_rate(difference)}"
                                      f"\ntolerance:           {format_rate(tol)}"))


    def testCmsSwap(self):
        """Testing Hagan-pricer flat-vol equivalence for swaps (lognormal case)."""
        print("Testing Hagan-pricer flat-vol equivalence for swaps (lognormal case)...")
        vars_ = self.vars

        swapIndex = vars_.swapIndexBase
        spread = 0.0
        swapLengths = [1, 5, 6, 10]
        cms_swaps = []

        for length in swapLengths:
            # Use MakeCms helper
            # Need to provide a coupon pricer during construction or set it later?
            # Let's assume we set it later.
            # MakeCms(swapTenor, swapIndex, iborIndex, iborSpread, forwardStart, cmsPricer)
            # The pricer seems mandatory. We'll create dummy/default and override in loop.
            # Default pricer needed:
            default_pricer = ql.AnalyticHaganPricer(vars_.atmVol, ql.GFunctionFactory.Standard,
                                                   ql.QuoteHandle(ql.SimpleQuote(0.0)))

            swap = ql.MakeCms(ql.Period(length, ql.Years),
                              swapIndex,
                              vars_.iborIndex, spread,
                              ql.Period(10, ql.Days), # Forward start
                              default_pricer) # Provide a default pricer
            cms_swaps.append(swap)

        tol = 2.0e-4

        for j in range(len(vars_.yieldCurveModels)):
            num_pricer = vars_.numericalPricers[j]
            ana_pricer = vars_.analyticPricers[j]

            num_pricer.setSwaptionVolatility(vars_.atmVol)
            ana_pricer.setSwaptionVolatility(vars_.atmVol)

            for i, swap in enumerate(cms_swaps):
                cms_leg = swap.leg(0) # Assuming CMS leg is leg 0

                # Price with numerical pricer
                ql.setCouponPricer(cms_leg, num_pricer)
                priceNum = swap.NPV()

                # Price with analytical pricer
                ql.setCouponPricer(cms_leg, ana_pricer)
                priceAn = swap.NPV()

                difference = abs(priceNum - priceAn)
                is_linear_tsr = (isinstance(num_pricer, ql.LinearTsrPricer))

                self.assertLessEqual(difference, tol,
                                     msg=(f"\nLength in Years:  {swapLengths[i]}"
                                          f"\nswap index:       {swapIndex.name()}"
                                          f"\nibor index:       {vars_.iborIndex.name()}"
                                          f"\nspread:           {format_rate(spread)}"
                                          f"\nYieldCurve Model: {vars_.yieldCurveModels[j]}"
                                          f"\nNumerical Pricer NPV: {priceNum:.6f}" # Use format_price if defined
                                          f"{' (Linear TSR Model)' if is_linear_tsr else ''}"
                                          f"\nAnalytic Pricer NPV:  {priceAn:.6f}"
                                          f"\ndifference:          {difference:.6f}"
                                          f"\ntolerance:           {tol:.6f}"))


    def testParity(self):
        """Testing put-call parity for capped-floored CMS coupons (lognormal case)."""
        print("Testing put-call parity for capped-floored CMS coupons (lognormal case)...")
        vars_ = self.vars

        # Use all volatility structures defined in CommonVars
        swaptionVols = [vars_.atmVol, vars_.SabrVolCube1, vars_.SabrVolCube2]

        swapIndex = vars_.swapIndexBase
        startDate = vars_.termStructure.referenceDate() + ql.Period(20, ql.Years)
        # Ensure payment date uses a valid calendar for advance
        paymentDate = swapIndex.fixingCalendar().advance(startDate, 1, ql.Years, ql.Following)
        endDate = paymentDate # For CMS coupon structure
        nominal = 1.0
        # infiniteCap = ql.nullDouble()
        # infiniteFloor = ql.nullDouble()
        gearing = 1.0
        spread = 0.0
        day_counter = vars_.iborIndex.dayCounter() # Use index day counter for coupon

        discount = vars_.termStructure.discount(paymentDate)

        # Base swaplet (plain CMS coupon)
        swaplet = ql.CmsCoupon(paymentDate, nominal, startDate, endDate,
                               swapIndex.fixingDays(), swapIndex, gearing, spread,
                               startDate, endDate, day_counter)

        strikes = [0.02, 0.07, 0.12] # Reduced range from C++ for brevity

        for strike in strikes:
            # Capped coupon
            caplet = ql.CappedFlooredCmsCoupon(paymentDate, nominal, startDate, endDate,
                                               swapIndex.fixingDays(), swapIndex, gearing, spread,
                                               [strike], [], # Cap strike, no floor
                                               startDate, endDate, day_counter)
            # Floored coupon
            floorlet = ql.CappedFlooredCmsCoupon(paymentDate, nominal, startDate, endDate,
                                                 swapIndex.fixingDays(), swapIndex, gearing, spread,
                                                 [], [strike], # No cap, floor strike
                                                 startDate, endDate, day_counter)

            for vol_handle in swaptionVols:
                for j in range(len(vars_.yieldCurveModels)):
                    num_pricer = vars_.numericalPricers[j]
                    ana_pricer = vars_.analyticPricers[j]

                    pricers = [num_pricer, ana_pricer]

                    for k, pricer in enumerate(pricers):
                        pricer.setSwaptionVolatility(vol_handle) # Set current vol structure

                        swaplet.setPricer(pricer)
                        caplet.setPricer(pricer)
                        floorlet.setPricer(pricer)

                        # Calculate prices (discounted values of the option parts)
                        swaplet_pv = swaplet.price() # PV of the CMS rate payment
                        caplet_pv = caplet.price() # PV of Max(CMS-K, 0) + PV(CMS) if capped? No, price is option value.
                        floorlet_pv = floorlet.price() # PV of Max(K-CMS, 0)

                        # Parity: Cap(K) - Floor(K) = Swaplet - K * Discount
                        # PV(Cap) - PV(Floor) = PV(Swaplet) - PV(K)
                        # PV(K) = nominal * accrualPeriod * strike * discount
                        accrualPeriod = swaplet.accrualPeriod() # Get accrual period from coupon
                        fixed_pv = nominal * accrualPeriod * strike * discount

                        # Check: caplet_pv - floorlet_pv == swaplet_pv - fixed_pv
                        difference = abs((caplet_pv - floorlet_pv) - (swaplet_pv - fixed_pv))

                        is_linear_tsr = (k == 0 and isinstance(pricer, ql.LinearTsrPricer)) # k=0 is numerical
                        tol = 1.0e-7 if is_linear_tsr else 2.0e-5 # Use tolerances from C++ test

                        pricer_type = "Numerical" if k == 0 else "Analytic"

                        self.assertLessEqual(difference, tol,
                                             msg=(f"\nParity Check Failed:"
                                                  f"\nCoupon payment date: {paymentDate}"
                                                  f"\nStrike:              {format_rate(strike)}"
                                                  f"\nYieldCurve Model:    {vars_.yieldCurveModels[j]}"
                                                  f"\nPricer Type:         {pricer_type}"
                                                  f"{' (Linear TSR Model)' if is_linear_tsr else ''}"
                                                  # f"\nVol Structure:     {vol_handle}" # Handle doesn't have useful name
                                                  f"\nPV(Caplet):          {caplet_pv:.8f}"
                                                  f"\nPV(Floorlet):        {floorlet_pv:.8f}"
                                                  f"\nPV(Swaplet):         {swaplet_pv:.8f}"
                                                  f"\nPV(Fixed Strike K):  {fixed_pv:.8f}"
                                                  f"\nCap - Floor:         {caplet_pv - floorlet_pv:.8f}"
                                                  f"\nSwaplet - PV(K):     {swaplet_pv - fixed_pv:.8f}"
                                                  f"\nDifference:          {difference:.8g}"
                                                  f"\nTolerance:           {tol:.1g}"))


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)